https://stockanalysis.com/stocks/

In [1]:
from datetime import datetime, timedelta
import pickle
import sys
from time import sleep

from bs4 import BeautifulSoup
from requests_html import HTMLSession

sys.path.append('..')
import yf_clone.yfinance as yf
#import yfinance as yf

sent: <object object at 0x130c3d140>


In [2]:
EXCHANGE_PATHS = {
    'nasdaq': 'nasdaq',
    'nyse': 'newyorkstockexchange',
    'amex': 'americanstockexchange'}
base_url = 'https://www.advfn.com'
session = HTMLSession()

In [3]:
def get_soup(url):
    try:
        res = session.get(url)
        html = res.html.html
        soup = BeautifulSoup(html, 'html.parser')
        return soup
    except BaseException as e:
        print(f'Problem retrieving page: {url}\n{e}')

In [4]:
def get_table(soup):
    try:
        table = soup.find('table', class_='market')
        if table is None:
            table = soup.find('table', class_='market tab1')
        return table
    except BaseExcetpion as e:
        print('Unable to obtain table')
        print(e)

In [5]:
def get_rows(table):
    try:
        rows = table.find_all('tr')
        rows = [
            row for row in rows if 'class' in row.attrs 
            and row.attrs['class'][0].startswith('ts')]
        return rows
    except BaseException as e:
        print('Unable to extract rows')
        print(e)

In [6]:
def is_valid(symbol):
    MAXLEN = 4
    return len(symbol) <= MAXLEN and symbol.isalpha()

In [7]:
def get_symbol(row):
    SYMBOL_COL_IDX = 1
    try:
        symbol = row.find_all('a')[SYMBOL_COL_IDX].text
        if is_valid(symbol):
            return symbol
    except BaseException as e:
        print('Unable to get symbol from row:', row)
        print(e)

In [8]:
def extract_symbols(soup):
    symbols = []
    table = get_table(soup)
    rows = get_rows(table)
    for row in rows:
        symbol = get_symbol(row)
        if symbol is not None:
            symbols.append(symbol)
    return symbols

In [9]:
def extract_all_symbols():
    all_symbols = []
    for exchange, path in EXCHANGE_PATHS.items():
        print(f'Beggining {exchange}\nLetter:')
        for letter in list('ABCDEFGHIJKLMNOPQRSTUVWXYZ'):
            print(letter, end='')
            url = f'{base_url}/{exchange}/{path}.asp?companies={letter}'
            soup = get_soup(url)
            symbols = extract_symbols(soup)
            all_symbols += symbols
        print()
    return sorted(all_symbols)

In [10]:
soup = get_soup('https://www.advfn.com/nasdaq/nasdaq.asp?companies=A')
#soup
get_table(soup)

In [11]:
all_symbols = extract_all_symbols()

Beggining nasdaq
Letter:
AUnable to extract rows
'NoneType' object has no attribute 'find_all'


TypeError: 'NoneType' object is not iterable

In [11]:
all_symbols = sorted(list(set(all_symbols)))

In [12]:
all_symbols[:10]

['A', 'AA', 'AAAP', 'AABA', 'AABC', 'AAC', 'AACC', 'AACE', 'AACI', 'AACQ']

In [13]:
all_symbols[-10:]

['ZWS', 'ZY', 'ZYBT', 'ZYME', 'ZYNE', 'ZYXI', 'ZZK', 'ZZO', 'ZZX', 'ZZZ']

In [14]:
with open('../data/all_symbols.pkl', 'wb') as f:
    pickle.dump(all_symbols, f)

In [15]:
n = len(all_symbols)
n

12009

In [21]:
END = datetime.now()
START = END - timedelta(7)
test = yf.download(['AAPL', 'TSLA', 'ZQZZ'], start=str(START.date()), end=str(END.date()))['Close']
test

[*********************100%***********************]  3 of 3 completed

1 Failed download:
['ZQZZ']: YFTzMissingError('possibly delisted; no timezone found')


Ticker,AAPL,TSLA,ZQZZ
Date,,,
2025-03-24,220.729996,278.390015,NaN
2025-03-25,223.750000,288.140015,NaN
2025-03-26,221.529999,272.059998,NaN
2025-03-27,223.850006,273.130005,NaN
2025-03-28,217.899994,263.549988,NaN


In [22]:
test2 = test.loc[:, test.iloc[-1, :].notnull()]
test2

Ticker,AAPL,TSLA
Date,,
2025-03-24,220.729996,278.390015
2025-03-25,223.750000,288.140015
2025-03-26,221.529999,272.059998
2025-03-27,223.850006,273.130005
2025-03-28,217.899994,263.549988


In [23]:
test2.columns

Index(['AAPL', 'TSLA'], dtype='object', name='Ticker')

In [24]:
start = str(START.date())
end = str(END.date())
start, end

('2025-03-23', '2025-03-30')

### Next Time
Code below is inefficient, bc it retries symbols if < $2 instead of removing them. 
Add to `omit` instead and do not rerun. (Update for all code chunks below.)

In [28]:
#i = 0
#active = []
#failed = []
#omit = []

while i <= n:
    try:
        first = i
        last = min(i + 100, n)
        batch = all_symbols[first:last]
        print(f'Batch: {batch[0]} - {batch[-1]}', end='\r')
        df = yf.download(batch, start=start, end=end)['Close']
        df = df.loc[:, df.iloc[-1, :].notnull()]
        batch_active = df.columns
        # Exclude if current val < 2.00
        df = df.loc[:, df.iloc[-1, :] >= 2.]
        batch_omit = [sym for sym in batch_active if sym not in df.columns]
        omit += list(batch_omit)
        active += list(df.columns)
        fails = [sym for sym in batch if sym not in df.columns and sym not in batch_omit]
        failed += fails
        sleep(12)
        i += 100
    except Exception as e:
        print(e)
        pass

[*********************100%***********************]  100 of 100 completed

53 Failed downloads:
['EMBB', 'EMKR', 'EIA', 'ELMG', 'ELBO', 'ELOQ', 'EMG', 'EGN', 'EMFT', 'EMCD', 'ELAM', 'EIHI', 'EIGR', 'EMHZ', 'ELEC', 'EMI', 'EJ', 'ELOY', 'ELTE', 'EMBX', 'ELNK', 'EMLB', 'ELU', 'EGT', 'EICU', 'ELRC']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-23 -> 2025-03-30)')
['EHT', 'EGOV', 'ELVT', 'EMBK', 'EGLX', 'EGIF', 'EHR', 'EGI', 'EMCI', 'ELXR', 'ELAT', 'EGLE', 'EIDX', 'EJFA', 'ELMS', 'EMCF', 'ELGX', 'EGGF', 'ELCC', 'EIGI', 'ELY', 'EMDA', 'ELJ']: YFTzMissingError('possibly delisted; no timezone found')
['EMLD', 'EGIO', 'ELYM', 'ELIQ']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-23 -> 2025-03-30) (Yahoo error = "No data found, symbol may be delisted")')
[**                     5%                       ]  5 of 100 completed

[*********************100%***********************]  100 of 100 completed

48 Failed downloads:
['EPIQ', 'EPIK', 'ENSI', 'EPEX', 'EMXX', 'EPHC', 'EMQ', 'EMSA', 'ENCY', 'ENTL', 'ENTU', 'EPMD', 'EMPW', 'EOCA', 'ENOC', 'EPIC', 'ENMC', 'EMSD', 'EPEN', 'EMSO', 'ENWV', 'EPNY', 'ENLC', 'ENH', 'ENZY', 'EPG', 'EPRE']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-23 -> 2025-03-30)')
['ENDP', 'ENV', 'ENJY']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-23 -> 2025-03-30) (Yahoo error = "No data found, symbol may be delisted")')
['ENFC', 'EMWP', 'ENT', 'ENVI', 'ENOB', 'ENTF', 'ENPC', 'ENNV', 'ENBA', 'ENER', 'EPWR', 'EOCW', 'ENCP', 'EPOC', 'EPHY', 'EPAY', 'EPZM', 'ENCX']: YFTzMissingError('possibly delisted; no timezone found')
[**                     4%                       ]  4 of 100 completed

[*********************100%***********************]  100 of 100 completed

50 Failed downloads:
['EQGP', 'ERYY', 'ESD', 'ESSF', 'EQY', 'ESIO', 'ESIC', 'ETE', 'ESCH', 'ERS', 'ETRM', 'ERGF', 'ERTS', 'ESND', 'ERT', 'ESUS', 'ESPD', 'ESRX', 'ETAK', 'ESST', 'ESLR']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-23 -> 2025-03-30)')
['ETAO', 'ETTX', 'ETRN', 'ERF']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-23 -> 2025-03-30) (Yahoo error = "No data found, symbol may be delisted")')
['ERES', 'ESTR', 'EQM', 'ESTE', 'ERA', 'EQRX', 'ESV', 'ESMT', 'EQOS', 'ESMK', 'ESBK', 'ETM', 'ESGC', 'EROS', 'EQD', 'ESM', 'ESSC', 'ETAC', 'ESAC', 'ESXB', 'ETFC', 'ERNB', 'ERI', 'ETEL', 'EQHA']: YFTzMissingError('possibly delisted; no timezone found')
[**                     5%                       ]  5 of 100 completed

[*********************100%***********************]  100 of 100 completed

41 Failed downloads:
['ETWC', 'EURO', 'EZFL', 'EVAC', 'EXJF', 'EXEA', 'EVVV', 'EVDY', 'EUBK', 'EXBD', 'EXAC', 'EUFS', 'EXAM', 'EZEM', 'EZT', 'EYET', 'EXXI', 'EVHC', 'EXCO', 'EXEE', 'EURX', 'EXA', 'EVBS', 'EVST']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-23 -> 2025-03-30)')
['EVE', 'EVK', 'EVBG', 'EXTO', 'EURN']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-23 -> 2025-03-30) (Yahoo error = "No data found, symbol may be delisted")')
['EVOJ', 'EYES', 'EVSI', 'EXD', 'EXN', 'EXTN', 'EUCR', 'EVOP', 'EXPR', 'EXPC', 'EUSG', 'EXFO']: YFTzMissingError('possibly delisted; no timezone found')
[**                     4%                       ]  4 of 100 completed

[*********************100%***********************]  100 of 100 completed

57 Failed downloads:
['FBEI', 'FBMT', 'FBMI', 'FACE', 'FAUS', 'FCEC', 'FBRC', 'FCFC', 'FDI', 'FAV', 'FCLF', 'FCFL', 'FCPO', 'FBST', 'FCS', 'FCFP', 'FADV', 'FDCC', 'FDML', 'FAOO', 'FCH', 'FBG', 'FBNW', 'FCSE', 'FBAY', 'FCGI']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-23 -> 2025-03-30)')
['FBCM', 'FABK', 'FCRD', 'FDEU', 'FDEF', 'FBNK', 'FALB', 'FCRZ', 'FCRW', 'FCAC', 'FDRY', 'FATP', 'FCEB', 'FCAX', 'FCSX', 'FCSC', 'FACA', 'FB', 'FBSS', 'FCBP', 'FCAU', 'FAZZ', 'FAII', 'FAZE', 'FDC', 'FBC', 'FBHS', 'FBM', 'FCEA']: YFTzMissingError('possibly delisted; no timezone found')
['FATH', 'FAM']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-23 -> 2025-03-30) (Yahoo error = "No data found, symbol may be delisted")')
[***                    6%                       ]  6 of 100 completed

...

[*********************100%***********************]  100 of 100 completed

52 Failed downloads:
['WSB', 'WSPT', 'WTBK', 'WSKI', 'WTRS', 'WSCI', 'WPTE', 'WTCO', 'WPZ', 'WPSC', 'WVCM', 'WSTC', 'WSKY', 'WTRX', 'WNR', 'WSSI', 'WSTF', 'WNNB', 'WSII', 'WR', 'WSBK', 'WSPI', 'WOFC', 'WTNY', 'WSBA', 'WSBI', 'WTEL', 'WRNC', 'WTHN', 'WRSP']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-23 -> 2025-03-30)')
['WTRH', 'WPF', 'WPCB', 'WRE', 'WP', 'WRI', 'WTRU', 'WRAC', 'WRTC', 'WORK', 'WSFG', 'WNTG', 'WSTG', 'WPCA', 'WRLS', 'WTR', 'WQGA', 'WPG', 'WTT', 'WNNR', 'WPX']: YFTzMissingError('possibly delisted; no timezone found')
['WRK']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-23 -> 2025-03-30) (Yahoo error = "No data found, symbol may be delisted")')
[                       0%                       ]

[*********************100%***********************]  100 of 100 completed

61 Failed downloads:
['WWMO', 'XLTC', 'XETA', 'XLBS', 'WWAY', 'XBKS', 'WZEN', 'XCYT', 'XF', 'XLKS', 'WWWB', 'WYPT', 'XE', 'XK', 'XLES', 'XIVH', 'XD', 'XICO', 'XFNE', 'WWVY', 'XMLG', 'WYN', 'WYOI', 'XNVA', 'XING', 'WWON', 'XNPT', 'XCRA', 'XLYS', 'XMSR', 'XLUS', 'XOXO', 'XLVS', 'WWAV', 'XATA', 'XH', 'XOCM', 'XNWK', 'XLIS', 'XLPS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-23 -> 2025-03-30)')
['XAN', 'XLUX', 'XPAX', 'WWE', 'XGTI', 'XPDI', 'WXCO', 'WWAC', 'XL', 'XON', 'XLRN', 'XPDB', 'XM', 'XEC', 'XENT', 'XFML', 'XIV', 'XOG', 'XLNX', 'WYND']: YFTzMissingError('possibly delisted; no timezone found')
['XFIN']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-23 -> 2025-03-30) (Yahoo error = "No data found, symbol may be delisted")')
[**                     5%                       ]  5 of 100 completed

[*********************100%***********************]  100 of 100 completed

58 Failed downloads:
['YAKC', 'XXIA', 'XSEL', 'XS', 'XSON', 'YDNT', 'YAVY', 'YESR', 'YDIV', 'XTXI', 'XRDC', 'YGRO', 'YZC', 'XPLR', 'YTHK', 'YANB', 'XUE', 'YZCM', 'YDIW', 'XRM', 'XTLY', 'XVIX', 'YOKU', 'YDKN', 'YXR', 'XRGC', 'YTEC', 'XYNY', 'XRIT', 'ZAIS', 'XTND', 'XRA', 'XU', 'XXV', 'XSBG', 'YCB']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-23 -> 2025-03-30)')
['YTPG', 'YELL', 'Y', 'XPOA', 'YUMA', 'YSAC', 'YPRO', 'XSPL', 'YGF', 'YAC', 'YRCW', 'YMTX', 'YHOO', 'YGYI', 'YGTY', 'ZAGG', 'XSPA', 'YOGA']: YFTzMissingError('possibly delisted; no timezone found')
['YVR', 'YNDX', 'YS', 'YTEN']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-23 -> 2025-03-30) (Yahoo error = "No data found, symbol may be delisted")')
[**                     4%                       ]  4 of 100 completed

[*********************100%***********************]  99 of 100 completed

54 Failed downloads:
['ZAPZ', 'ZSAN', 'ZEN', 'ZNH', 'ZING', 'ZAYO', 'ZME', 'ZGNX', 'ZFGN', 'ZEV', 'ZDIV', 'ZN', 'ZBZX', 'ZT', 'ZGYH', 'ZNGA', 'ZEST', 'ZVO', 'ZIXI', 'ZIOP', 'ZWRK', 'ZARS', 'ZHIC']: YFTzMissingError('possibly delisted; no timezone found')
['ZONA', 'ZLTQ', 'ZPIN', 'ZOMX', 'ZEPE', 'ZRAN', 'ZONS', 'ZVXI', 'ZKID', 'ZVV', 'ZOO', 'ZIGO', 'ZOLT', 'ZICA', 'ZIPR', 'ZRBA', 'ZLRG', 'ZMBA', 'ZNDT', 'ZOES', 'ZENX', 'ZOLL', 'ZILA', 'ZILG', 'ZOOG', 'ZUO', 'ZFC']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-23 -> 2025-03-30)')
['ZNTE', 'ZLS', 'ZFOX']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-23 -> 2025-03-30) (Yahoo error = "No data found, symbol may be delisted")')
['ZTST']: YFInvalidPeriodError("ZTST: Period 'max' is invalid, must be of the format 1d, 5d, etc.")
[**********************56%**                     ]  5 of 9 completed

[*********************100%***********************]  9 of 9 completed

5 Failed downloads:
['ZZK', 'ZZX', 'ZZO']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-23 -> 2025-03-30)')
['ZY', 'ZYNE']: YFTzMissingError('possibly delisted; no timezone found')


In [29]:
'ACAI' in active, 'ACAI' in failed, 'ACAI' in omit 

(False, False, True)

### 2nd Attempt

In [30]:
len(active), len(omit)

(4663, 1170)

In [31]:
n = len(failed)
n

6176

In [32]:
failed_again = []
i = 0
while i <= n:
    try:
        first = i
        last = min(i + 100, n)
        batch = failed[first:last]
        print(f'Batch: {batch[0]} - {batch[-1]}', end='\r')
        df = yf.download(batch, start=start, end=end)['Adj Close']
        print(df.shape)
        if len(df):
            df = df.loc[:, df.iloc[-1, :].notnull()]
            batch_active = df.columns
            # Exclude if current val < 2.00
            df = df.loc[:, df.iloc[-1, :] >= 2.]
            batch_omit = [sym for sym in batch_active if sym not in df.columns]
            omit += list(batch_omit)
            active += list(df.columns)
            fails = [sym for sym in batch if sym not in df.columns and sym not in batch_omit]
            failed_again += fails
        else:
            failed_again += batch
        sleep(3)
        i += 100
    except Exception as e:
        print(e)
        pass

[**                     5%                       ]  5 of 100 completed

[*********************100%***********************]  100 of 100 completed

100 Failed downloads:
['ACFC', 'AANB', 'ABNJ', 'ABCO', 'ABCD', 'ABBC', 'ACTI', 'ACAP', 'ABCW', 'ABBK', 'ACCI', 'ABAC', 'ABVA', 'AACE', 'ABGX', 'ACTY', 'ACTS', 'AAAP', 'ACG', 'ABBI', 'ABXA', 'ACCL', 'ADBL', 'ABLX', 'AATI', 'ABAX', 'ACAS', 'ABRX', 'AATK', 'ABER', 'ACOM', 'AABC', 'ACRN', 'AACC', 'ACLA', 'ACGY', 'ABRI', 'ACW', 'ABTL', 'ACCP', 'ACXM', 'AAII', 'ACAT', 'AAPC', 'ACEC', 'ACLI']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-23 -> 2025-03-30)')
['ACND', 'ACHN', 'ABST', 'ACEV', 'ACTD', 'AAU', 'ADF', 'ACII', 'AABA', 'ACAM', 'ACQR', 'ACGN', 'AAWW', 'ADEX', 'ABB', 'ACH', 'ACTT', 'ADES', 'ABTX', 'AAIN', 'ACDI', 'ACAX', 'ACIA', 'ADAL', 'ACAH', 'ACCT', 'ACTC', 'ACY', 'AACQ', 'ABGI', 'AAXN', 'ABIL', 'ACAQ', 'ACBI', 'ABC', 'AAC', 'ACRO', 'ACOR', 'AAQC', 'ABDC', 'AAIC', 'ACRX', 'ACC', 'ABII']: YFTzMissingError('possibly delisted; no timezone found')
['ACAC', 'ACAB', 'ACER', 'AAMC', 'ABIO'

(0, 100)


[*                      2%                       ]  2 of 100 completed

[*********************100%***********************]  100 of 100 completed

100 Failed downloads:
['AFCO', 'AINN', 'ADZA', 'ADIN', 'AETC', 'ADIC', 'AHS', 'ADRX', 'ADSX', 'AHCI', 'AFOP', 'ADLR', 'AFAM', 'AEOS', 'AEGR', 'AF', 'AFBC', 'AIB', 'AFCI', 'ADK', 'AET', 'AFSI', 'AGU', 'AHGP', 'ADSC', 'AFBK', 'ADTK', 'AEPI', 'AGCC', 'AFFX', 'AIRM', 'AGAM', 'AHP', 'ADPI', 'AIRN', 'AHPI', 'AE', 'AGII']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-23 -> 2025-03-30)')
['AIPC', 'AEHA', 'AESC', 'AGLE', 'AEY', 'AEGN', 'AGFS', 'AFH', 'AHPA', 'AFIN', 'AHAC', 'ADMS', 'AHRN', 'AGIL', 'AESE', 'AIPT', 'AGCB', 'AIRV', 'AGC', 'AHC', 'ADS', 'ADRO', 'AGAC', 'AETI', 'AFAQ', 'AIMC', 'AIMT', 'AGE', 'AFI', 'AERI', 'ADGI', 'AIC', 'AFTR', 'AERC', 'AIKI', 'ADMP', 'ADOM', 'AEAC', 'AGGR', 'ADRA', 'AGTC', 'AHM', 'ADSW', 'AINV', 'AGN', 'AFGH', 'ADOC', 'ADGF']: YFTzMissingError('possibly delisted; no timezone found')
['AEL', 'AGTI', 'AGR', 'AFAR', 'AFAC', 'AEZS', 'ADTH', 'AGBA', 'AFT', 'AIRC', 'AI

...

[*********************100%***********************]  100 of 100 completed

100 Failed downloads:
['XLTC', 'YAKC', 'WWMO', 'XXIA', 'XETA', 'XS', 'XSON', 'WWAY', 'XSEL', 'YAVY', 'YDNT', 'XCYT', 'XBKS', 'XLBS', 'WZEN', 'YESR', 'XLKS', 'XF', 'WWWB', 'XE', 'YDIV', 'WYPT', 'YGRO', 'XRDC', 'XTXI', 'XK', 'XD', 'XPLR', 'XLES', 'XIVH', 'XICO', 'XUE', 'YANB', 'XFNE', 'XMLG', 'YDIW', 'WWVY', 'XRM', 'XTLY', 'WYN', 'XRGC', 'XVIX', 'WYOI', 'YDKN', 'XYNY', 'XRIT', 'XING', 'XU', 'XTND', 'XNVA', 'WWON', 'XNPT', 'XRA', 'XCRA', 'XLYS', 'XMSR', 'XLUS', 'XOXO', 'XH', 'XOCM', 'XLVS', 'XLPS', 'YCB', 'XNWK', 'XSBG', 'XATA', 'XLIS', 'XXV']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-23 -> 2025-03-30)')
['XFIN', 'YNDX']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-23 -> 2025-03-30) (Yahoo error = "No data found, symbol may be delisted")')
['XAN', 'XLUX', 'YELL', 'XPAX', 'WWE', 'XGTI', 'XPDI', 'XPOA', 'Y', 'WXCO', 'XON', 'XL', 'XLRN', 'XSPL', 'YGF', 'YAC

(0, 100)


[**                     4%                       ]  3 of 76 completed

[*********************100%***********************]  75 of 76 completed

76 Failed downloads:
['ZRBA', 'ZONS', 'ZZK', 'YTHK', 'ZOLL', 'ZVXI', 'YZCM', 'ZENX', 'ZIPR', 'ZRAN', 'ZZX', 'ZZO', 'ZILA', 'ZLRG', 'ZKID', 'ZLTQ', 'ZOO', 'ZICA', 'ZONA', 'ZMBA', 'ZILG', 'ZNDT', 'ZVV', 'ZPIN', 'YOKU', 'ZOOG', 'ZIGO', 'YXR', 'ZUO', 'ZEPE', 'ZOMX', 'ZOLT', 'YTEC', 'ZFC', 'YZC', 'ZOES', 'ZAIS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-23 -> 2025-03-30)')
['ZNH', 'ZME', 'ZIXI', 'ZSAN', 'ZN', 'YTPG', 'ZIOP', 'ZFGN', 'ZY', 'ZAGG', 'ZWRK', 'ZARS', 'ZEN', 'ZGNX', 'ZYNE', 'ZAPZ', 'YRCW', 'ZBZX', 'ZT', 'ZEV', 'YOGA', 'YUMA', 'ZHIC', 'ZING', 'YPRO', 'ZVO', 'ZGYH', 'ZAYO', 'YSAC', 'ZDIV', 'ZEST', 'ZNGA']: YFTzMissingError('possibly delisted; no timezone found')
['YVR', 'ZFOX', 'ZNTE', 'YS', 'YTEN', 'ZLS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-23 -> 2025-03-30) (Yahoo error = "No data found, symbol may be delisted")')
['ZTST']: YFInvalidPerio

(0, 76)


### 3rd Attempt

In [33]:
'ACAI' in active, 'ACAI' in failed, 'ACAI' in omit 

(False, False, True)

In [34]:
len(active), len(omit)

(4663, 1170)

In [35]:
failed = failed_again[:]
n = len(failed)
n

6176

In [36]:
failed_again = []
i = 0
while i <= n :
    try:
        first = i
        last = min(i + 100, n)
        batch = failed[first:last]
        print(f'Batch: {batch[0]} - {batch[-1]}', end='\r')
        df = yf.download(batch, start=start, end=end)['Adj Close']
        print(df.shape)
        if len(df):
            df = df.loc[:, df.iloc[-1, :].notnull()]
            batch_active = df.columns
            # Exclude if current val < 2.00
            df = df.loc[:, df.iloc[-1, :] >= 2.]
            batch_omit = [sym for sym in batch_active if sym not in df.columns]
            omit += list(batch_omit)
            active += list(df.columns)
            fails = [sym for sym in batch if sym not in df.columns and sym not in batch_omit]
            failed_again += fails
        else:
            failed_again += batch
        sleep(3)
        i += 100
    except Exception as e:
        print(e)
        pass

[**                     5%                       ]  5 of 100 completed

[*********************100%***********************]  100 of 100 completed

100 Failed downloads:
['ACFC', 'ABCO', 'ABBC', 'ABNJ', 'AANB', 'ABCD', 'ACAP', 'ACTI', 'ABCW', 'ABBK', 'ACCI', 'ABAC', 'AACE', 'ACG', 'ABVA', 'ABGX', 'ACTS', 'AAAP', 'ACTY', 'ACCL', 'ABBI', 'ABXA', 'ADBL', 'ABLX', 'ACAS', 'AATI', 'ABAX', 'ABRX', 'AATK', 'ABER', 'ACRN', 'AABC', 'AACC', 'ACOM', 'ACLA', 'ACGY', 'ABRI', 'ABTL', 'ACW', 'ACCP', 'ACXM', 'AAII', 'ACAT', 'AAPC', 'ACEC', 'ACLI']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-23 -> 2025-03-30)')
['ACND', 'ABST', 'ACTD', 'ACEV', 'ACHN', 'AAU', 'ACII', 'ADF', 'AABA', 'ACQR', 'ACAM', 'ACGN', 'AAWW', 'ADEX', 'ACH', 'ABB', 'ACAX', 'ADES', 'ACTT', 'AAIN', 'ABTX', 'ACDI', 'ACIA', 'ADAL', 'ACTC', 'ACY', 'ACAH', 'ACCT', 'AACQ', 'ACAQ', 'ABGI', 'ABC', 'ABIL', 'AAXN', 'ACBI', 'ACRO', 'AAC', 'ACOR', 'ABDC', 'AAIC', 'AAQC', 'ABII', 'ACC', 'ACRX']: YFTzMissingError('possibly delisted; no timezone found')
['ACAC', 'ACAB', 'ACER', 'AAMC', 'ABIO'

(0, 100)


[                       0%                       ]

[*********************100%***********************]  100 of 100 completed

100 Failed downloads:
['AINN', 'AFCO', 'ADZA', 'ADIN', 'ADIC', 'AETC', 'AHS', 'ADSX', 'ADRX', 'AHCI', 'AFOP', 'AFAM', 'ADLR', 'AF', 'AEGR', 'AEOS', 'AFBC', 'AIB', 'ADK', 'AFCI', 'AET', 'AFSI', 'AGU', 'AHGP', 'AFBK', 'ADTK', 'ADSC', 'AEPI', 'AFFX', 'AGCC', 'AIRM', 'AGAM', 'AHP', 'AIRN', 'ADPI', 'AE', 'AHPI', 'AGII']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-23 -> 2025-03-30)')
['AEL', 'AGTI', 'AGR', 'AFAC', 'AFAR', 'AEZS', 'ADTH', 'AGBA', 'AFT', 'AIRC', 'AIF', 'AGRX', 'ADRT', 'AINC']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-23 -> 2025-03-30) (Yahoo error = "No data found, symbol may be delisted")')
['AIPC', 'AESC', 'AEHA', 'AGLE', 'AEY', 'AEGN', 'AFH', 'AHPA', 'AFIN', 'AGFS', 'ADMS', 'AHAC', 'AHRN', 'AGIL', 'AESE', 'AIPT', 'AIRV', 'AGCB', 'AGC', 'ADS', 'AHC', 'ADRO', 'AGAC', 'AETI', 'AFAQ', 'AGE', 'AIMC', 'AIMT', 'AFI', 'AERI', 'AERC', 'AIC', 'ADGI

...

[*********************100%***********************]  100 of 100 completed

100 Failed downloads:
['XXIA', 'XLTC', 'YAKC', 'XETA', 'WWMO', 'XSEL', 'WWAY', 'XSON', 'XS', 'YDNT', 'XLBS', 'XBKS', 'WZEN', 'YAVY', 'XCYT', 'WWWB', 'XLKS', 'YESR', 'XF', 'XE', 'WYPT', 'YDIV', 'XK', 'YGRO', 'XTXI', 'XRDC', 'XPLR', 'XD', 'XLES', 'XICO', 'XIVH', 'XUE', 'YANB', 'XMLG', 'XFNE', 'WWVY', 'XRM', 'YDIW', 'XRGC', 'XTLY', 'WYOI', 'YDKN', 'XVIX', 'WYN', 'XRIT', 'XYNY', 'XTND', 'XING', 'XRA', 'XU', 'XNVA', 'XLYS', 'WWON', 'XCRA', 'XNPT', 'XLUS', 'XMSR', 'XOXO', 'YCB', 'XH', 'XOCM', 'XLVS', 'XSBG', 'XATA', 'XLIS', 'XXV', 'XNWK', 'XLPS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-23 -> 2025-03-30)')
['XAN', 'XLUX', 'YELL', 'XPDI', 'WWE', 'XPAX', 'XGTI', 'WXCO', 'Y', 'XPOA', 'XL', 'XON', 'XLRN', 'XSPL', 'YGF', 'YAC', 'XPDB', 'YMTX', 'YGYI', 'XEC', 'XM', 'YHOO', 'YGTY', 'XENT', 'XSPA', 'XFML', 'XOG', 'XIV', 'XLNX', 'WYND']: YFTzMissingError('possibly delisted; no timezone found')


(0, 100)


[**                     4%                       ]  3 of 76 completed

[*********************100%***********************]  75 of 76 completed

76 Failed downloads:
['ZZK', 'ZONS', 'ZRBA', 'YTHK', 'ZOLL', 'ZVXI', 'ZENX', 'YZCM', 'ZIPR', 'ZZX', 'ZRAN', 'ZZO', 'ZLRG', 'ZKID', 'ZILA', 'ZICA', 'ZOO', 'ZLTQ', 'ZONA', 'YOKU', 'ZILG', 'ZPIN', 'ZVV', 'ZMBA', 'ZNDT', 'YXR', 'ZOOG', 'ZIGO', 'ZUO', 'ZOMX', 'ZEPE', 'ZOLT', 'YTEC', 'ZFC', 'YZC', 'ZOES', 'ZAIS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-23 -> 2025-03-30)')
['ZME', 'ZNH', 'ZIXI', 'ZN', 'ZIOP', 'ZSAN', 'YTPG', 'ZY', 'ZAGG', 'ZYNE', 'ZEN', 'ZGNX', 'ZWRK', 'ZFGN', 'ZARS', 'ZAPZ', 'ZBZX', 'ZT', 'YRCW', 'ZEV', 'YOGA', 'YUMA', 'ZHIC', 'ZGYH', 'ZAYO', 'ZING', 'YPRO', 'ZVO', 'ZDIV', 'ZEST', 'ZNGA', 'YSAC']: YFTzMissingError('possibly delisted; no timezone found')
['ZFOX', 'YVR', 'ZNTE', 'YS', 'YTEN', 'ZLS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-23 -> 2025-03-30) (Yahoo error = "No data found, symbol may be delisted")')
['ZTST']: YFInvalidPerio

(0, 76)


### 4th Attempt

In [37]:
len(active), len(omit)

(4663, 1170)

In [38]:
failed = failed_again[:]
n = len(failed)
n

6176

In [39]:
with open('../data/all_symbols.pkl', 'wb') as f:
    pickle.dump(active, f)

In [32]:
'INTI' in active, 'INTI' in omit

(False, True)

In [33]:
'TSLA' in active, 'TSLA' in omit

(True, False)

In [ ]:
!say complete

In [ ]:
# Run...?
'''
failed_again = []
i = 0
while i <= n - 100:
    first = i
    last = min(i + 100, n)
    batch = failed[first:last]
    print(f'Batch: {batch[0]} - {batch[-1]}', end='\r')
    df = yf.download(batch, start=start, end=end)['Adj Close']
    print(df.shape)
    if len(df):
        df = df.loc[:, df.iloc[-1, :].notnull()]
        batch_active = df.columns
        # Exclude if current val < 2.00
        df = df.loc[:, df.iloc[-1, :] >= 2.]
        batch_omit = [sym for sym in batch_active if sym not in df.columns]
        omit += list(batch_omit)
        active += list(df.columns)
        fails = [sym for sym in batch if sym not in df.columns and sym not in batch_omit]
        failed_again += fails
    else:
        failed_again += batch
    sleep(3)
    i += 100
'''